In [ ]:
import loompy
import scvelo as scv
import pandas

In [ ]:
files = ['/home/edoardo/prova_v/loom/embryo_mut.loom', '/home/edoardo/prova_v/loom/P2_Mut_hSETBP1.loom']
filename = "/home/edoardo/prova_v/loom/all_Mut.loom"
loompy.combine(files, filename, key="Accession")

In [ ]:
filename = '/home/edoardo/prova_v/loom/all_ctrl.loom'
adata = scv.read(filename, cache=True)
adata.var_names_make_unique()

In [ ]:
adata.obs_names

In [ ]:
df = pandas.read_csv('/home/edoardo/prova_v/meta_all_ctrl.csv')
df

In [ ]:
df['Label_cluster'].value_counts()

In [ ]:
adata.obs['seurat'] = 'filtered'
adata.obs['cell_name'] = 'tobeassigned'
for i in range(len(adata.obs.index)):
    adata.obs['cell_name'][i] = adata.obs.index[i].replace('P2_Ctrl_hSETBP1:','P2_ctrl_').replace('x','').replace('embryo_ctrl:','embryo_ctrl_')
adata.obs

In [ ]:
# need to speed up this one
for j in range(len(adata.obs['cell_name'])):
    for i in range(len(df['Unnamed: 0'])):
        if df['Unnamed: 0'][i] == adata.obs['cell_name'][j]:
            adata.obs['seurat'][j] = df['Label_cluster'][i]
            break
adata.obs

In [ ]:
adata.obs['seurat'].value_counts()

In [ ]:
adata_filt = adata[adata.obs['seurat'] != 'filtered',:]

In [ ]:
adata_filt.obs

In [ ]:
adata_filt_seurat.obs['clusters'] = adata_filt_seurat.obs['seurat']
scv.pl.proportions(adata_filt_seurat)

In [ ]:
scv.pp.filter_and_normalize(adata_filt, n_top_genes=4000, enforce=True)
scv.pp.moments(adata_filt, n_pcs=30, n_neighbors=30)

In [ ]:
adata_filt.obs

In [ ]:
scv.tl.velocity(adata_filt)

In [ ]:
scv.tl.velocity_graph(adata_filt, n_jobs=20)

In [ ]:
scv.tl.umap(adata_filt)

In [ ]:
palette = ['#00BFC4','#7CAE00','grey','#00BE67','#C77CFF','#FF61CC','black','#00A9FF','#CD9600','pink','#F8766D']
scv.pl.umap(adata_filt, color=['seurat'], legend_loc='right_margin', palette=palette)

In [ ]:
scv.pl.velocity_embedding_stream(adata_filt, basis='umap', color=['seurat'], palette = palette, dpi=100, legend_loc='right_margin')

In [ ]:
scv.tl.velocity_confidence(adata_filt)
scv.pl.scatter(adata_filt, c='velocity_length', cmap='coolwarm', perc=[5, 95])

In [ ]:
adata_filt.obsm['X_umap']

In [ ]:
adata_filt_seurat = adata_filt.copy()

In [ ]:
ucoord = scv.load('UMAP_all_ctrl.csv')
#ucoord.index = [ x.rstrip('-1_2') for x in ucoord.index ]
ucoord = ucoord.reindex(index=adata_filt_seurat.obs['cell_name'].values)
ucoord = ucoord.to_numpy()
#for i in ucoord.index:
#    ucoord['Unnamed: 0'][i] = ucoord['Unnamed: 0'][i].rstrip('-1_2')
#ucoord.index = ucoord['Unnamed: 0'].values
#ucoord = ucoord.reindex(index=adata_filt_seurat.obs['cell_name'].index)
#ucoord = ucoord.drop('Unnamed: 0', axis='columns')

In [ ]:
adata_filt_seurat.obsm['X_umap'] = ucoord
adata_filt_seurat.obsm['X_umap']

In [ ]:
palette = ["#BE0032", "#875692", "#848482", "#A1CAF1", "#E68FAC", "brown", "#222222", "#F3C300", "#F38400", "#008856", "#C2B280"
]
scv.pl.umap(adata_filt_seurat, color=['seurat'], palette = palette, legend_loc='right_margin')

In [ ]:
palette =  ["#BE0032", "#875692", "#848482", "#A1CAF1", "#E68FAC", "brown", "#222222", "#F3C300", "#F38400", "#008856", "#C2B280"
]
scv.tl.velocity(adata_filt_seurat, n_jobs=20)
scv.tl.velocity_graph(adata_filt_seurat)
scv.pl.velocity_embedding_stream(adata_filt_seurat, basis='umap', color=['seurat'], palette = palette, 
                                 legend_loc='right_margin', save='all_ctrl_velo.svg')
#save='all_ctrl_velo_5.svg'

In [ ]:
g2m_g = pandas.read_csv('/home/edoardo/prova_v/g2m.genes.csv')['g2m.genes']
s_g = pandas.read_csv('/home/edoardo/prova_v/s.genes.csv')['s.gene']
scv.tl.score_genes_cell_cycle(adata_filt_seurat, g2m_genes= g2m_g, s_genes= s_g)
scv.pl.scatter(adata_filt_seurat, color_gradients=['S_score', 'G2M_score'], smooth=True, perc=[5, 95])

In [ ]:
scv.tl.rank_velocity_genes(adata_filt_seurat, groupby='seurat', min_corr=.3)
df = scv.DataFrame(adata_filt_seurat.uns['rank_velocity_genes']['names'])
df.head()

In [ ]:
df['AP_RGC'][1:5]


In [ ]:
palette = ['#00BFC4','#7CAE00','grey','#00BE67','#C77CFF','#FF61CC','brown','#00A9FF','#CD9600','pink','#F8766D']
scv.pl.velocity(adata_filt_seurat, ['Nfib'], ncols=2, color=['seurat'], 
                palette = palette)

In [ ]:
scv.tl.recover_dynamics(adata_filt_seurat, n_jobs=20)

In [ ]:
scv.tl.velocity_graph(adata_filt_seurat, n_jobs=20)

In [ ]:
scv.tl.latent_time(adata_filt_seurat)
scv.pl.scatter(adata_filt_seurat, color='latent_time', color_map='gnuplot', size=80)

In [ ]:
top_genes = adata_filt_seurat.var['fit_likelihood'].sort_values(ascending=False).index[:300]
scv.pl.heatmap(adata_filt_seurat, var_names=top_genes, 
               sortby='latent_time', col_color='seurat', 
               n_convolve=100, palette=palette)

In [ ]:
top_genes = adata_filt_seurat.var['fit_likelihood'].sort_values(ascending=False).index
scv.pl.scatter(adata_filt_seurat, basis=top_genes[:15], 
               ncols=5, frameon=False, color=['seurat'],
              palette=palette)

In [ ]:
var_names = ['Sox6']
scv.pl.scatter(adata_filt_seurat,var_names, frameon=False, color=['seurat'], palette=palette)
scv.pl.scatter(adata_filt_seurat, x='latent_time', y=var_names, projection='2d', 
               frameon=False, color=['seurat'], palette=palette)

In [ ]:
scv.tl.rank_dynamical_genes(adata_filt_seurat, groupby='seurat')
df = scv.get_df(adata_filt_seurat, 'rank_dynamical_genes/names')
df.head(5)

In [ ]:
df.to_csv("/home/edoardo/prova_v/loom/Mut_genes_in_velo.csv",index=False)

In [ ]:
scv.tl.velocity_confidence(adata_filt_seurat)
keys = 'velocity_length', 'velocity_confidence'
scv.pl.scatter(adata_filt_seurat, c=keys, cmap='coolwarm', perc=[5, 95])
#save='MUT_velocity.svg'

In [ ]:
df = adata_filt_seurat.obs.groupby('seurat')[keys].mean().T
df.style.background_gradient(cmap='coolwarm', axis=1)
#df.to_csv("/home/edoardo/prova_v/loom/MUT_velo_in_clusters.csv",index=False)

In [ ]:
scv.pl.velocity_graph(adata_filt_seurat, threshold=.1)

In [ ]:
x, y = scv.utils.get_cell_transitions(adata_filt_seurat, basis='umap', starting_cell=70)
ax = scv.pl.velocity_graph(adata_filt_seurat, c='lightgrey', edge_width=.05, show=False)
ax = scv.pl.scatter(adata_filt_seurat, x=x, y=y, s=120, c='ascending', cmap='gnuplot', ax=ax)

In [ ]:
scv.tl.velocity_pseudotime(adata_filt_seurat)
scv.pl.scatter(adata_filt_seurat, color='velocity_pseudotime', cmap='gnuplot')
#save='all_mut_velo_corrected.svg'

In [ ]:
# PAGA requires to install igraph, if not done yet.
#!pip install python-igraph --upgrade --quiet

In [ ]:
# this is needed due to a current bug - bugfix is coming soon.
adata_filt_seurat.uns['neighbors']['distances'] = adata_filt_seurat.obsp['distances']
adata_filt_seurat.uns['neighbors']['connectivities'] = adata_filt_seurat.obsp['connectivities']

scv.tl.paga(adata_filt_seurat, groups='seurat')
df = scv.get_df(adata_filt_seurat, 'paga/transitions_confidence', precision=2).T
df.style.background_gradient(cmap='Blues').format('{:.2g}')

In [ ]:
df.to_csv("/home/edoardo/prova_v/loom/PAGA_CTRL.csv",index=False)

In [ ]:
scv.pl.paga(adata_filt_seurat, basis='umap', size=50, alpha=.1,
            min_edge_width=2, node_size_scale=1, threshold=.25)
#save = '/home/edoardo/prova_v/figures/PAGA_mut_filt.svg'